# Cloning Dataset Repositories

In [1]:
!git clone https://gitlab.com/shimorina/webnlg-dataset.git

Cloning into 'webnlg-dataset'...
remote: Enumerating objects: 5112, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 5112 (delta 2), reused 0 (delta 0), pack-reused 5106 (from 1)
Receiving objects: 100% (5112/5112), 26.09 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (4010/4010), done.
Updating files: 100% (1425/1425), done.


In [2]:
!git clone https://github.com/WebNLG/webnlg_toolkit.git
%cd webnlg_toolkit
%ls
!pip install -e .

Cloning into 'webnlg_toolkit'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 230 (delta 45), reused 211 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (230/230), 16.78 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/webnlg_toolkit
LICENSE  README.md  requirements.txt  setup.py  webnlg_toolkit/
Obtaining file:///content/webnlg_toolkit
  Preparing metadata (setup.py) ... done
  Running setup.py develop for webnlg_toolkit


# Importing Libraries

In [3]:
import spacy
import re
from collections import defaultdict

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
import random

In [6]:
import os
import glob
import json
import numpy as np
import pandas as pd
import torch
import re
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.metrics import precision_score, recall_score
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    AdamW,
    get_linear_schedule_with_warmup
)
!pip install datasets
from datasets import Dataset as HFDataset
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import webnlg_toolkit
from webnlg_toolkit.utils.data import load_webnlg_dataset
from webnlg_toolkit.utils.data import load_webnlg_xml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.7 MB/s eta 0:00:00


# Preprocessing Data

In [7]:
dataset_root = "/content/webnlg-dataset/release_v3.0/en/train/"
test_root = "/content/webnlg-dataset/release_v3.0/en/dev/"
final_file = "/content/webnlg-dataset/release_v3.0/en/test/rdf-to-text-generation-test-data-with-refs-en.xml"

# Function to find dataset files
def find_dataset_files(root_folder, file_extension=".xml"):
    dataset_files = []
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(file_extension):
                dataset_files.append(os.path.join(subdir, file))
    return dataset_files

# Load training data
def load_all_data(root_folder):
    dataset_files = find_dataset_files(root_folder, file_extension=".xml")
    all_data = []
    for file in dataset_files:
        print(f"Loading: {file}")
        data = load_webnlg_dataset(file, task="rdf2text")
        # print(data)
        all_data.extend(data)
    return all_data

# Load final data
def load_final_data(file):
    all_data = []
    print(f"Loading: {file}")
    data = load_webnlg_dataset(file, task="rdf2text")
    # print(data)
    all_data.extend(data)
    return all_data

# Load training and test data
train_data = load_all_data(dataset_root)
test_data = load_all_data(test_root)
final_data = load_final_data(final_file)

# Convert to DataFrames
train_df = pd.DataFrame(train_data, columns=["input", "output"])
test_df = pd.DataFrame(test_data, columns=["input", "output"])
final_df = pd.DataFrame(final_data, columns=["input", "output"])
print(f"Training examples: {len(train_df)}")
print(f"Test examples: {len(test_df)}")
print(f"Final examples: {len(final_df)}")

# Display some examples
print("\nTraining examples:")
print(train_df.head(3))

print("\nTest examples:")
print(test_df.head(3))

print("\nFinal examples:")
print(final_df.head(3))

Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/CelestialBody_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/MeanOfTransportation_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/Athlete_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/Food_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/Astronaut_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/University_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/Artist_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/City_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/WrittenWork_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/Company_allSolutions.xml
Loading: /content/webnlg-dataset/release_v3.0/en/train/1triples/Politician_allSolutions.x

In [8]:
def extract_relations(input_text):
    """Extract relation types from input text in the format <S> entity1 <P> relation <O> entity2"""
    relations = []
    pattern = r'<P>\s+(.*?)\s+<O>'
    matches = re.findall(pattern, input_text)
    return matches

# Apply relation extraction to create a new column with relation types
train_df['relations'] = train_df['input'].apply(extract_relations)
test_df['relations'] = test_df['input'].apply(extract_relations)
final_df['relations'] = final_df['input'].apply(extract_relations)


In [9]:
def extract_entities(input_text):
    """Extract entity pairs from input text in the format <S> entity1 <P> relation <O> entity2"""
    entity_pairs = []
    pattern = r'<S>\s+(.*?)\s+<P>.*?<O>\s+(.*?)(?:\n|$)'
    matches = re.findall(pattern, input_text)
    return matches

# Apply entity extraction to create a new column with entity pairs
train_df['entity_pairs'] = train_df['input'].apply(extract_entities)
test_df['entity_pairs'] = test_df['input'].apply(extract_entities)
final_df['entity_pairs'] = final_df['input'].apply(extract_entities)

In [10]:
def get_entity_type(entity_text):
    """
    Determine entity type using SpaCy NER and heuristics.
    Returns one of: PERSON, LOCATION, ORGANIZATION, DATE, MISC
    """
    # Process with SpaCy
    doc = nlp(entity_text)

    # Check if SpaCy found any entities
    if doc.ents:
        # Get the most common entity type
        ent_type = doc.ents[0].label_

        # Map SpaCy entity types to our simplified types
        if ent_type in ["PERSON", "PER"]:
            return "PERSON"
        elif ent_type in ["LOC", "GPE", "FAC"]:
            return "LOCATION"
        elif ent_type in ["ORG"]:
            return "ORGANIZATION"
        elif ent_type in ["DATE", "TIME"]:
            return "DATE"

    # Use heuristics as backup
    lower_text = entity_text.lower()

    # Check for locations
    location_words = ["city", "country", "state", "river", "mountain", "ocean", "sea", "lake"]
    if any(word in lower_text for word in location_words):
        return "LOCATION"

    # Check for organizations
    org_indicators = ["inc", "corp", "llc", "company", "association", "university", "institute"]
    if any(word in lower_text for word in org_indicators):
        return "ORGANIZATION"

    # Check for date patterns
    if re.search(r'\b(19|20)\d{2}\b', entity_text) or \
       re.search(r'\b(January|February|March|April|May|June|July|August|September|October|November|December)\b', entity_text, re.IGNORECASE):
        return "DATE"

    # Default to MISC
    return "MISC"

In [11]:
# Make a list of all unique relations
all_relations = []
for relations in train_df['relations']:
    all_relations.extend(relations)

# Sort the unique relations and save them into unique_relations
# Then create a mapping dictionary from relation names to numerical IDs
# And reverse mapping from IDs back to relation names
unique_relations = sorted(list(set(all_relations)))
relation_to_id = {relation: idx for idx, relation in enumerate(unique_relations)}
id_to_relation = {idx: relation for relation, idx in relation_to_id.items()}

print(f"\nNumber of unique relation types: {len(unique_relations)}")
print("Example relation types:", unique_relations[:5])


Number of unique relation types: 372
Example relation types: ['1st runway length feet', '1st runway length metre', '1st runway number', '1st runway surface type', '2nd runway surface type']


In [12]:
# Define a PyTorch Dataset class for relation extraction tasks that iscontent aware of entity types
class ContentAwareRelationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, entity_markers, entity_end_markers, max_length=128, relation_to_id=None):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.relation_to_id = relation_to_id
        self.entity_markers = entity_markers
        self.entity_end_markers = entity_end_markers

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]

        # Get relation and entity pair
        relation = row['relations'][0] if len(row['relations']) > 0 else "unknown"
        entity_pair = row['entity_pairs'][0] if len(row['entity_pairs']) > 0 else ("", "")
        entity1, entity2 = entity_pair

        # Get context (use output text if available)
        context = row.get('output', "")

        # Determine entity types
        entity1_type = get_entity_type(entity1)
        entity2_type = get_entity_type(entity2)

        # Get type-specific markers
        e1_start = self.entity_markers.get(entity1_type, self.entity_markers["DEFAULT"])
        e1_end = self.entity_end_markers.get(entity1_type, self.entity_end_markers["DEFAULT"])

        e2_start = self.entity_markers.get(entity2_type, self.entity_markers["DEFAULT"])
        e2_end = self.entity_end_markers.get(entity2_type, self.entity_end_markers["DEFAULT"])

        # Format with type-specific markers
        if context:
            # Try to find entities in context
            formatted_text = context

            # Escape regex special characters in entities
            e1_esc = re.escape(entity1)
            e2_esc = re.escape(entity2)

            # Replace entities with marked versions - handle longer entity first to avoid substring issues
            if len(entity1) >= len(entity2):
                formatted_text = re.sub(f"\b{e1_esc}\b", f"{e1_start} {entity1} {e1_end}", formatted_text)
                formatted_text = re.sub(f"\b{e2_esc}\b", f"{e2_start} {entity2} {e2_end}", formatted_text)
            else:
                formatted_text = re.sub(f"\b{e2_esc}\b", f"{e2_start} {entity2} {e2_end}", formatted_text)
                formatted_text = re.sub(f"\b{e1_esc}\b", f"{e1_start} {entity1} {e1_end}", formatted_text)

            # If entities weren't found in context, append them with markers
            if e1_start not in formatted_text:
                formatted_text += f" {e1_start} {entity1} {e1_end}"
            if e2_start not in formatted_text:
                formatted_text += f" {e2_start} {entity2} {e2_end}"
        else:
            # No context - just use the marked entities
            formatted_text = f"{e1_start} {entity1} {e1_end} {e2_start} {entity2} {e2_end}"

        # Tokenize
        encoding = self.tokenizer(
            formatted_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Remove batch dimension
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}

        # Add relation label
        if self.relation_to_id:
            relation_id = self.relation_to_id.get(relation, 0)
            encoding['labels'] = torch.tensor(relation_id, dtype=torch.long)

        return encoding


In [13]:
# Define entity type markers
entity_markers = {
    "PERSON": "[PERSON]",
    "LOCATION": "[LOCATION]",
    "ORGANIZATION": "[ORG]",
    "DATE": "[DATE]",
    "MISC": "[MISC]",
    "DEFAULT": "[ENTITY]"
}

entity_end_markers = {
    "PERSON": "[/PERSON]",
    "LOCATION": "[/LOCATION]",
    "ORGANIZATION": "[/ORG]",
    "DATE": "[/DATE]",
    "MISC": "[/MISC]",
    "DEFAULT": "[/ENTITY]"
}

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Add special tokens for entity markers
special_tokens = list(entity_markers.values()) + list(entity_end_markers.values())
special_tokens_dict = {'additional_special_tokens': special_tokens}
tokenizer.add_special_tokens(special_tokens_dict)

print(f"Added {len(special_tokens)} special tokens to tokenizer")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Added 12 special tokens to tokenizer


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the saved model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=372
)
model.resize_token_embeddings(len(tokenizer))

# Load saved weights
model_path = "/content/NerBert.pt"
model.load_state_dict(torch.load(model_path, map_location = device))
model.to(device)
model.eval()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
<ipython-input-14-a4eed96b7305>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipp

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30534, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [15]:
# Create datasets
train_dataset = ContentAwareRelationDataset(
    train_df,
    tokenizer,
    entity_markers,
    entity_end_markers,
    max_length=128,
    relation_to_id=relation_to_id
)
test_dataset = ContentAwareRelationDataset(
    test_df,
    tokenizer,
    entity_markers,
    entity_end_markers,
    max_length=128,
    relation_to_id=relation_to_id
)
final_dataset = ContentAwareRelationDataset(
    final_df,
    tokenizer,
    entity_markers,
    entity_end_markers,
    max_length=128,
    relation_to_id=relation_to_id
)

# Split training data into train and validation
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
generator1 = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size], generator=generator1)

print(f"\nTraining set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")
print(f"Final set size: {len(final_dataset)}")


Training set size: 31883
Validation set size: 3543
Test set size: 4464
Final set size: 5150


In [16]:
# Create data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
final_loader = DataLoader(final_dataset, batch_size=batch_size)

## Evaluate Model

In [17]:
# Evaluation function
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            predictions = torch.argmax(logits, dim=-1)

            total_loss += loss.item()

            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(batch["labels"].cpu().numpy())

    # Calculate metrics
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
    f1 = f1_score(all_labels, all_preds, average='weighted')
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)

    return total_loss / len(dataloader), accuracy, f1, precision, recall, all_preds, all_labels

In [18]:
final_loss, final_accuracy, final_f1, final_precision, final_recall, _, _ = evaluate(model, final_loader, device)
print(f"Validation loss: {final_loss:.4f}")
print(f"Validation accuracy: {final_accuracy:.4f}")
print(f"Validation F1 score: {final_f1:.4f}")
print(f"Validation precision: {final_precision:.4f}")
print(f"Validation recall: {final_recall:.4f}")

Evaluating:   0%|          | 0/322 [00:00<?, ?it/s]

Validation loss: 2.9329
Validation accuracy: 0.5718
Validation F1 score: 0.5013
Validation precision: 0.4817
Validation recall: 0.5718


# Real Time Inference

In [19]:
# Extract the relation between two entities using the trained BERT model.
def extract_relation(entity1, entity2, text=None):
    encoding = tokenizer(
        entity1,
        entity2,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )
    encoding = {k: v.to(device) for k, v in encoding.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()
    relation = id_to_relation.get(prediction, "unknown")

    return relation

# Test with Entity Input

In [20]:
# Enter your entity here (Example usage provided)
examples = [
    ("Barack Obama", "United States"),
    ("Apple", "iPhone"),
    ("Paris", "France"),
    ("Albert Einstein", "1955")
]
entity1 = "AmeriGas"
entity2 = "United States"

In [21]:
bert_relation = extract_relation(entity1, entity2)

print(f"Entities: '{entity1}' and '{entity2}'")
print(f"BERT predicted relation: '{bert_relation}'")

Entities: 'AmeriGas' and 'United States'
BERT predicted relation: 'region served'
